In [98]:
import pandas as pd
import numpy as np

In [99]:
# Read in sales.csv as a data frame

df = pd.read_csv('sales.csv')
df

Account                          Name            Rep        Manager  \
0    714466               Trantow-Barrows   Craig Booker   Debra Henley   
1    714466               Trantow-Barrows   Craig Booker   Debra Henley   
2    714466               Trantow-Barrows   Craig Booker   Debra Henley   
3    737550  Fritsch, Russel and Anderson   Craig Booker   Debra Henley   
4    146832                  Kiehn-Spinka  Daniel Hilton   Debra Henley   
5    218895                     Kulas Inc  Daniel Hilton   Debra Henley   
6    218895                     Kulas Inc  Daniel Hilton   Debra Henley   
7    412290                 Jerde-Hilpert     John Smith   Debra Henley   
8    740150                    Barton LLC     John Smith   Debra Henley   
9    141962                    Herman LLC    Cedric Moss  Fred Anderson   
10   163416                   Purdy-Kunde    Cedric Moss  Fred Anderson   
11   239344                    Stokes LLC    Cedric Moss  Fred Anderson   
12   239344                    Stokes LLC    Cedric Moss  Fred Anderson   
13   307599   Kassulke, Ondricka and Metz     Wendy Yule  Fred Anderson   
14   688981                   Keeling LLC     Wendy Yule  Fred Anderson   
15   729833                     Koepp Ltd     Wendy Yule  Fred Anderson   
16   729833                     Koepp Ltd     Wendy Yule  Fred Anderson   

        Product  Quantity   Price     Status  
0           CPU         1   30000  presented  
1      Software         1   10000  presented  
2   Maintenance         2    5000    pending  
3           CPU         1   35000   declined  
4           CPU         2   65000        won  
5           CPU         2   40000    pending  
6      Software         1   10000  presented  
7   Maintenance         2    5000    pending  
8           CPU         1   35000   declined  
9           CPU         2   65000        won  
10          CPU         1   30000  presented  
11  Maintenance         1    5000    pending  
12     Software         1   10000  presented  
13  Maintenance         3    7000        won  
14          CPU         5  100000        won  
15          CPU         2   65000   declined  
16      Monitor         2    5000  presented

# Pivot Table 
## Setting the index
The simplest pivot table must have:
* a dataframe
* an index 

Let’s use Name as our index.

In [100]:
pd.pivot_table(df,index=["Name"])

Account     Price  Quantity
Name                                                      
Barton LLC                    740150.0   35000.0  1.000000
Fritsch, Russel and Anderson  737550.0   35000.0  1.000000
Herman LLC                    141962.0   65000.0  2.000000
Jerde-Hilpert                 412290.0    5000.0  2.000000
Kassulke, Ondricka and Metz   307599.0    7000.0  3.000000
Keeling LLC                   688981.0  100000.0  5.000000
Kiehn-Spinka                  146832.0   65000.0  2.000000
Koepp Ltd                     729833.0   35000.0  2.000000
Kulas Inc                     218895.0   25000.0  1.500000
Purdy-Kunde                   163416.0   30000.0  1.000000
Stokes LLC                    239344.0    7500.0  1.000000
Trantow-Barrows               714466.0   15000.0  1.333333

Take note:
* Only numerical columns will be aggregated.
* What do the numbers in the Price column mean? How do you think they were generated? The same goes for Account and Quantity columns. Is there a way to change how they are generated?

## Multiple indexing
How about indexing on multiple columns? Let's set both the Manager and Rep as the index. 

In [101]:
pd.pivot_table(df,index=["Manager","Rep"])

Account         Price  Quantity
Manager       Rep                                            
Debra Henley  Craig Booker   720237.0  20000.000000  1.250000
              Daniel Hilton  194874.0  38333.333333  1.666667
              John Smith     576220.0  20000.000000  1.500000
Fred Anderson Cedric Moss    196016.5  27500.000000  1.250000
              Wendy Yule     614061.5  44250.000000  3.000000

Do the numbers in the Quantity column make sense to you? Once again, how was the Price column generated?

What we really want to take a look at next is Price. So, the Account and Quantity columns aren’t useful. Let’s remove them and explicitly define the 'Price' column by using the values field.

In [102]:
pd.pivot_table(df, 
               index = ["Manager","Rep"],
               values = ["Price"])

Price
Manager       Rep                 
Debra Henley  Craig Booker   20000
              Daniel Hilton  38333
              John Smith     20000
Fred Anderson Cedric Moss    27500
              Wendy Yule     44250

## Aggfunc
By default, the means in the 'Price' column is generated. We can do a sum by using aggfunc with np.sum.

In [103]:
pd.pivot_table(df,
               index = ["Manager","Rep"],
               values = ["Price"],
               aggfunc = np.sum)

Price
Manager       Rep                  
Debra Henley  Craig Booker    80000
              Daniel Hilton  115000
              John Smith      40000
Fred Anderson Cedric Moss    110000
              Wendy Yule     177000

Columns are optional. They are an additional way to segment the values you care about. But remember, the aggregation functions are applied to the values you list.

In [104]:
pd.pivot_table(df,
               index = ["Manager","Rep"],
               values = ["Price"],
               columns = ["Product"],
               aggfunc = [np.sum])

sum                             
                                Price                             
Product                           CPU Maintenance Monitor Software
Manager       Rep                                                 
Debra Henley  Craig Booker    65000.0      5000.0     NaN  10000.0
              Daniel Hilton  105000.0         NaN     NaN  10000.0
              John Smith      35000.0      5000.0     NaN      NaN
Fred Anderson Cedric Moss     95000.0      5000.0     NaN  10000.0
              Wendy Yule     165000.0      7000.0  5000.0      NaN

Let's replace the NaNs with 0.

In [105]:
pd.pivot_table(df,index = ["Manager","Rep"],
               values = ["Price"],
               columns = ["Product"],
               aggfunc = [np.sum],
               fill_value = 0) #fill NaNs with zero

sum                             
                              Price                             
Product                         CPU Maintenance Monitor Software
Manager       Rep                                               
Debra Henley  Craig Booker    65000        5000       0    10000
              Daniel Hilton  105000           0       0    10000
              John Smith      35000        5000       0        0
Fred Anderson Cedric Moss     95000        5000       0    10000
              Wendy Yule     165000        7000    5000        0

In [106]:
# Quantity might be useful, let's add it to the values list. 

pd.pivot_table(df,
               index = ["Manager","Rep"],
               values = ["Price","Quantity"],
               columns = ["Product"],
               aggfunc = [np.sum],
               fill_value = 0) #fill NaNs with zero

sum                                        \
                              Price                              Quantity   
Product                         CPU Maintenance Monitor Software      CPU   
Manager       Rep                                                           
Debra Henley  Craig Booker    65000        5000       0    10000        2   
              Daniel Hilton  105000           0       0    10000        4   
              John Smith      35000        5000       0        0        1   
Fred Anderson Cedric Moss     95000        5000       0    10000        3   
              Wendy Yule     165000        7000    5000        0        7   

                                                          
                                                          
Product                     Maintenance Monitor Software  
Manager       Rep                                         
Debra Henley  Craig Booker            2       0        1  
              Daniel Hilton           0       0        1  
              John Smith              2       0        0  
Fred Anderson Cedric Moss             1       0        1  
              Wendy Yule              3       2        0

To get a different take on things, let's remove Product from the columns and add it to the index. 

In [107]:
pd.pivot_table(df,
               index = ["Manager","Rep","Product"],
               values = ["Price","Quantity"],
               aggfunc = [np.sum,np.mean])

sum              mean         
                                          Price Quantity    Price Quantity
Manager       Rep           Product                                       
Debra Henley  Craig Booker  CPU           65000        2  32500.0      1.0
                            Maintenance    5000        2   5000.0      2.0
                            Software      10000        1  10000.0      1.0
              Daniel Hilton CPU          105000        4  52500.0      2.0
                            Software      10000        1  10000.0      1.0
              John Smith    CPU           35000        1  35000.0      1.0
                            Maintenance    5000        2   5000.0      2.0
Fred Anderson Cedric Moss   CPU           95000        3  47500.0      1.5
                            Maintenance    5000        1   5000.0      1.0
                            Software      10000        1  10000.0      1.0
              Wendy Yule    CPU          165000        7  82500.0      3.5
                            Maintenance    7000        3   7000.0      3.0
                            Monitor        5000        2   5000.0      2.0

## Margins

The margins argument controls whether the grand total of the columns are generated. If 'margins = True', an "All" row will be generated.

In [114]:
pd.pivot_table(df,
               index = ["Manager","Rep"],
               values = ["Price"],
               aggfunc = [np.sum,np.mean],
               margins = True)

sum          mean
                                Price         Price
Manager       Rep                                  
Debra Henley  Craig Booker    80000.0  20000.000000
              Daniel Hilton  115000.0  38333.000000
              John Smith      40000.0  20000.000000
Fred Anderson Cedric Moss    110000.0  27500.000000
              Wendy Yule     177000.0  44250.000000
All                          522000.0  30705.882353

In [116]:
pd.pivot_table(df,
               index = ["Manager","Rep"],
               values = ["Price"],
               aggfunc = [np.sum,np.mean],
               margins = False) # default is False, no need to specify

sum   mean
                              Price  Price
Manager       Rep                         
Debra Henley  Craig Booker    80000  20000
              Daniel Hilton  115000  38333
              John Smith      40000  20000
Fred Anderson Cedric Moss    110000  27500
              Wendy Yule     177000  44250